In [4]:
import requests
import json
import pandas as pd
from getpass import getpass
import time
import os
import folium
import pandas as pd
import getpass

In [5]:
token = getpass.getpass(prompt='Enter your token:')

Enter your token:········


In [8]:
# ZIP code SF 94108 94102 94105

def foursquare_places(venue):
    url = f"https://api.foursquare.com/v3/places/search?ll=37.790847%2C-122.4066358&radius=1500&categories=18006"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    
    df = pd.DataFrame(response["results"])
    
    return df


In [9]:
df = foursquare_places("Airport")

In [190]:
df = df.apply(lambda x: {
    'id': x['categories'][0]['id'],
    'categorie': x['categories'][0]['name'],
    'distance': x['distance'],
    'latitude': x['geocodes']['main']['latitude'],
    'longitude': x['geocodes']['main']['longitude'],
    'name': x['name'],
}, axis=1, result_type='expand')

In [10]:
# Create a Folium map centered on San Francisco 

df_sf = pd.read_csv("venues_near_office.csv")

sanfrancisco_map = folium.Map(location=[37.790847, -122.406635], zoom_start=12)

def create_icon(id):
    
     #NIGHT CLUB - COCKTAIL BAR - PUB
    if id == 10032:
        return folium.Icon(color='pink', icon='beer', prefix='fa')
    
     #BASKETBALL
    elif id == 18008:
        return folium.Icon(color='orange', icon='basketball-ball', prefix='fa') 
     
     #STARBUCKS
    elif id == 13035:
        return folium.Icon(color='green', icon='coffee', prefix='fa') 
    
    #AIRPORT
    elif id == 19040:
        return folium.Icon(color='purple', icon='plane', prefix='fa') 
    
    #TRANSPORT
    elif id == 19046:
        return folium.Icon(color='black', icon='subway', prefix='fa') 
    
      #Education
    elif id in [12058, 12059, 12057]:
        return folium.Icon(color='blue', icon='graduation-cap', prefix='fa') 
    
    #Veggie
    elif id in [13099, 13377, 13034, 13065, 13068, 13297, 13072]:
        return folium.Icon(color='lightgreen', icon='carrot', prefix='fa') 
    
    #Pet grooming service
    elif id == 11134:
        return folium.Icon(color='lightgray', icon='paw', prefix='fa')   
    
    #OTHERS?   
    else:
        return folium.Icon(color='white', icon='briefcase', prefix='fa')

# Add markers to map
for index, row in df_sf.iterrows():
    icon = create_icon(row["id"])
    
    new_marker = folium.Marker(location=[row["latitude"], row["longitude"]], 
                               icon = icon, 
                               tooltip = f'{row["name"]}<br>{row["categorie"]}')
    
    new_marker.add_to(sanfrancisco_map)

# Add radius circle
folium.Circle(location=[37.787872, -122.403612], radius=400, color='darkgreen', fill=True, fill_color='darkgreen').add_to(sanfrancisco_map)
folium.Circle(location=[37.787872, -122.403612], radius=800, color='blue', fill=True, fill_color='blue').add_to(sanfrancisco_map)
folium.Circle(location=[37.787872, -122.403612], radius=1500, color='red', fill=True, fill_color='red').add_to(sanfrancisco_map)
  

sanfrancisco_map